In [1]:
library('SelectSim')
library('tidyverse')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
set.seed(42)

In [3]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1     dplyr_1.1.4      
 [5] purrr_1.0.2       readr_2.1.5       tidyr_1.3.0       tibble_3.2.1     
 [9] ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

loaded via a nam

# Functions

In [4]:
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

# Read metadata and the pan-can run data for MSK

In [5]:
all_merged_metadata<- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/all_merged_data_genie_v15.rds')

In [6]:
msk_p_run_data <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_msk_primary_run_data_v15.rds')
msk_m_run_data <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_msk_meta_run_data_v15.rds')

In [7]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples<-readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/genie/msk_one_patient_one_sample_final.rds')

# MSK

In [8]:
str(msk_p_run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. ..$ truncating: num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	27825 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. .. ..$ mutation: num [1:27825] 4 4 1 0 4 2 3 3 2 3 ..

In [12]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter <- genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples %>% 
filter(SEQ_ASSAY_ID %in% c('MSK-IMPACT468','MSK-IMPACT505'))

In [13]:
sample_count<-genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>%
  group_by(final_class, SAMPLE_TYPE) %>%
  summarise(sample_count = n()) %>%
  filter(SAMPLE_TYPE %in% c("Primary", "Metastasis")) %>%
  group_by(final_class) %>%
  filter(all(sample_count >= 30))

`summarise()` has grouped output by 'final_class'. You can override using the `.groups` argument.


In [14]:
dim(genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter)

[1] 33306    22

In [15]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% count(SEQ_ASSAY_ID)

SEQ_ASSAY_ID,n
<chr>,<int>
MSK-IMPACT468,21672
MSK-IMPACT505,11634


In [17]:
length(unique(sample_count$final_class))

[1] 50

In [18]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter<-genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>%
  filter(final_class %in% sample_count$final_class)

In [19]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% head(2)

,PATIENT_ID,SEX,PRIMARY_RACE,ETHNICITY,CENTER,INT_CONTACT,INT_DOD,YEAR_CONTACT,DEAD,YEAR_DEATH,⋯,ONCOTREE_CODE,SAMPLE_TYPE,SEQ_ASSAY_ID,CANCER_TYPE,CANCER_TYPE_DETAILED,SAMPLE_TYPE_DETAILED,class,Tumor_run_group,final_class,final_run_group
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,GENIE-MSK-P-0014666,Female,White,Non-Spanish/non-Hispanic,MSK,11687,11687,2018,TRUE,2018,⋯,ACRM,Metastasis,MSK-IMPACT468,Melanoma,Acral Melanoma,Metastasis site unspecified,ACRM,ACRM,ACRM,Melanoma
2,GENIE-MSK-P-0014884,Male,White,Non-Spanish/non-Hispanic,MSK,20758,20758,2016,TRUE,2016,⋯,ACRM,Metastasis,MSK-IMPACT468,Melanoma,Acral Melanoma,Metastasis site unspecified,ACRM,ACRM,ACRM,Melanoma


In [20]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% count(SAMPLE_TYPE)

SAMPLE_TYPE,n
<chr>,<int>
Metastasis,11102
Primary,20376


In [22]:
msk_run_gene_list <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/data/msk_oncokb_gene_run_list.rds')

In [23]:
str(msk_run_gene_list)

List of 6
 $ all   : chr [1:202] "MTOR" "NRAS" "NTRK1" "SDHC" ...
 $ p_505 : chr [1:327] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_468 : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_410 : chr [1:282] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_341 : chr [1:249] "MTOR" "NRAS" "RIT1" "NTRK1" ...
 $ p_heme: chr [1:267] "MTOR" "NRAS" "CD58" "NTRK1" ...


In [24]:
consider_genes<-intersect(msk_run_gene_list$p_505,msk_run_gene_list$p_468)

In [25]:
length(consider_genes)

[1] 306

In [26]:
str(msk_p_run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. ..$ truncating: num [1:396, 1:27825] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	27825 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:27825] "GENIE-MSK-P-0000223-T01-IM3" "GENIE-MSK-P-0000679-T01-IM3" "GENIE-MSK-P-0000748-T01-IM3" "GENIE-MSK-P-0001408-T01-IM3" ...
  .. .. ..$ mutation: num [1:27825] 4 4 1 0 4 2 3 3 2 3 ..

## Create Equal Sample size primary and met sampling runs

In [27]:
genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% count(SAMPLE_TYPE)

SAMPLE_TYPE,n
<chr>,<int>
Metastasis,11102
Primary,20376


In [28]:
primary_samples<-unique((genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% filter(SAMPLE_TYPE=='Primary'))$SAMPLE_ID)
met_samples<-unique((genie_metadata_15_msk_with_mutation_selected_final_atleast_100_samples_filter %>% filter(SAMPLE_TYPE!='Primary'))$SAMPLE_ID)

In [29]:
msk_p_filter_run_data<- filter_run_data(run_data = msk_p_run_data,gene_order = consider_genes,sample_order = primary_samples)
msk_m_filter_run_data<- filter_run_data(run_data = msk_m_run_data,gene_order = consider_genes,sample_order = met_samples)

In [30]:
str(msk_p_filter_run_data)
str(msk_m_filter_run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:306, 1:20376] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  .. .. .. ..$ : chr [1:20376] "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0014893-T01-IM6" "GENIE-MSK-P-0018569-T01-IM6" "GENIE-MSK-P-0020908-T01-IM6" ...
  .. ..$ truncating: num [1:306, 1:20376] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  .. .. .. ..$ : chr [1:20376] "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0014893-T01-IM6" "GENIE-MSK-P-0018569-T01-IM6" "GENIE-MSK-P-0020908-T01-IM6" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	20376 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:20376] "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0014893-T01-IM6" "GENIE-MSK-P-0018569-T01-IM6" "GENIE-MSK-P-0020908-T01-IM6" ...
  .. .. ..$ mutation: num [1:20376] 2 1 1 1 0 4 

In [31]:
table(msk_p_filter_run_data$sample.class)


   ACRM    ACYC    ANSC    APAD    BLCA    CCOV    CESC    CHOL  CRC_HM  CRC_LM 
     32      49      83      48    1080      90     101     104     531    3079 
   DDLS    EHCH    ESCA    GBAD     GBC    GIST   HGSFT   HGSOC    HNSC     IDC 
    107      57     635      60      44     273      88     567     179    1973 
   IHCH     ILC   LGSOC     LMS    LUAD    LUNE    LUSC    MAAP     MEL     MFH 
    308     228      40     102    2365      40     419      42     367     118 
   MGCT     NBL   NSCLC      OS    PAAD   PANET    PRAD     RCC SARCNOS    SCLC 
     45     113     200      52    1576      91    1559     407     173     161 
   STAD    THPA    THPD UCEC_HM UCEC_LM     UCS    ULMS      UM     USC    UTUC 
    324     126      72     486     992     205      79      33     308     165 

In [32]:
table(msk_m_filter_run_data$sample.class)


   ACRM    ACYC    ANSC    APAD    BLCA    CCOV    CESC    CHOL  CRC_HM  CRC_LM 
     41      70      35      51     158      38      46      79      41    1043 
   DDLS    EHCH    ESCA    GBAD     GBC    GIST   HGSFT   HGSOC    HNSC     IDC 
     48      30     151      40      42      82      47     905     155    1750 
   IHCH     ILC   LGSOC     LMS    LUAD    LUNE    LUSC    MAAP     MEL     MFH 
     67     174      48      96    1342      49     148      56     762      48 
   MGCT     NBL   NSCLC      OS    PAAD   PANET    PRAD     RCC SARCNOS    SCLC 
     30      68     183      39     913      96     793     213      72     178 
   STAD    THPA    THPD UCEC_HM UCEC_LM     UCS    ULMS      UM     USC    UTUC 
    103     148      55      79     196      60      63      48      93      30 

In [33]:
sample_count_80_pct <- round(pmin(table(msk_p_filter_run_data$sample.class),table(msk_m_filter_run_data$sample.class))*.8)

In [34]:
sample_count_80_pct


   ACRM    ACYC    ANSC    APAD    BLCA    CCOV    CESC    CHOL  CRC_HM  CRC_LM 
     26      39      28      38     126      30      37      63      33     834 
   DDLS    EHCH    ESCA    GBAD     GBC    GIST   HGSFT   HGSOC    HNSC     IDC 
     38      24     121      32      34      66      38     454     124    1400 
   IHCH     ILC   LGSOC     LMS    LUAD    LUNE    LUSC    MAAP     MEL     MFH 
     54     139      32      77    1074      32     118      34     294      38 
   MGCT     NBL   NSCLC      OS    PAAD   PANET    PRAD     RCC SARCNOS    SCLC 
     24      54     146      31     730      73     634     170      58     129 
   STAD    THPA    THPD UCEC_HM UCEC_LM     UCS    ULMS      UM     USC    UTUC 
     82     101      44      63     157      48      50      26      74      24 

In [35]:
create_sample_list <- function(sample_count_80_pct,actual_data,times=10){
    sample_list <- c()
    for(i in c(1:times)){
        samples<-c()
        for(tumor in names(sample_count_80_pct)){
            samples<-c(samples,sample(names(which(actual_data==tumor)),sample_count_80_pct[tumor]))
        }
        sample_list[[i]]<-actual_data[samples]
    }
    return(sample_list)
}

In [36]:
msk_p_sampling_run_samples <- create_sample_list(sample_count_80_pct = sample_count_80_pct,actual_data = msk_p_filter_run_data$sample.class)
msk_m_sampling_run_samples <- create_sample_list(sample_count_80_pct = sample_count_80_pct,actual_data = msk_m_filter_run_data$sample.class)

In [37]:
length(msk_p_sampling_run_samples)
length(msk_m_sampling_run_samples)

[1] 10

[1] 10

In [38]:
msk_p_sample_run <- list()
msk_m_sample_run <- list()
for(i in c(1:10)){
    msk_p_sample_run[[i]]<-filter_run_data(run_data = msk_p_filter_run_data,gene_order = consider_genes,sample_order = names(msk_p_sampling_run_samples[[i]]))
    msk_m_sample_run[[i]]<-filter_run_data(run_data = msk_m_filter_run_data,gene_order = consider_genes,sample_order = names(msk_m_sampling_run_samples[[i]]))
}

In [ ]:
saveRDS(msk_p_sample_run,file='../data/msk_p_sample_run_data_list.rds')
saveRDS(msk_m_sample_run,file='../data/msk_m_sample_run_data_list.rds')

In [39]:
combine_run_data <- function(run_data1,run_data2,gene_order,sample_order1,sample_order2){
    final_run_data<-run_data1
    final_run_data$M$M$missense<-cbind(run_data1$M$M$missense[gene_order,sample_order1],run_data2$M$M$missense[gene_order,sample_order2])
    final_run_data$M$M$truncating<-cbind(run_data1$M$M$truncating[gene_order,sample_order1],run_data2$M$M$truncating[gene_order,sample_order2])
    final_run_data$alteration.class<- run_data1$alteration.class[gene_order]
    final_run_data$sample.class<-c(run_data1$sample.class[sample_order1],run_data2$sample.class[sample_order2])
    final_run_data$M$tmb$missense<-rbind(run_data1$M$tmb$missense[sample_order1,],run_data2$M$tmb$missense[sample_order2,])
    final_run_data$M$tmb$truncating<-rbind(run_data1$M$tmb$truncating[sample_order1,],run_data2$M$tmb$truncating[sample_order2,])
    return(final_run_data)
}

In [40]:
sample_count_80_pct_half<-round(sample_count_80_pct/2)

In [41]:
sample_count_80_pct_primary<- sample_count_80_pct-round(sample_count_80_pct/2)

In [42]:
sum(sample_count_80_pct_half+sample_count_80_pct_primary)

[1] 8195

In [43]:
sum(sample_count_80_pct)

[1] 8195

In [44]:
msk_p_sampling_run_samples <- create_sample_list(sample_count_80_pct = sample_count_80_pct_primary,actual_data = msk_p_filter_run_data$sample.class,times = 20)
msk_m_sampling_run_samples <- create_sample_list(sample_count_80_pct = sample_count_80_pct_half,actual_data = msk_m_filter_run_data$sample.class,times = 20)

In [45]:
combine_run_sample_run<-list()
for(i in c(1:20)){
    combine_run_sample_run[[i]]<-combine_run_data(run_data1= msk_p_filter_run_data,run_data2 = msk_m_filter_run_data,gene_order = consider_genes,sample_order1 = names(msk_p_sampling_run_samples[[i]]),
                                                  sample_order2 =names(msk_m_sampling_run_samples[[i]]) )
}

In [46]:
str(combine_run_sample_run,max.level = 1)

List of 20
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3
 $ :List of 3


In [47]:
table(combine_run_sample_run[[1]]$sample.class)


   ACRM    ACYC    ANSC    APAD    BLCA    CCOV    CESC    CHOL  CRC_HM  CRC_LM 
     26      39      28      38     126      30      37      63      33     834 
   DDLS    EHCH    ESCA    GBAD     GBC    GIST   HGSFT   HGSOC    HNSC     IDC 
     38      24     121      32      34      66      38     454     124    1400 
   IHCH     ILC   LGSOC     LMS    LUAD    LUNE    LUSC    MAAP     MEL     MFH 
     54     139      32      77    1074      32     118      34     294      38 
   MGCT     NBL   NSCLC      OS    PAAD   PANET    PRAD     RCC SARCNOS    SCLC 
     24      54     146      31     730      73     634     170      58     129 
   STAD    THPA    THPD UCEC_HM UCEC_LM     UCS    ULMS      UM     USC    UTUC 
     82     101      44      63     157      48      50      26      74      24 

In [ ]:
saveRDS(combine_run_sample_run,file='../data/msk_p_m_sample_run_data_list.rds')